https://www.kaggle.com/c/cs5785-fall19-final/

# Create an Ensemble Model

#### Get the Descriptions and Tags from File

In [1]:
import os
import numpy as np
import gensim
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd

In [2]:
def parse_descriptions(data_dir, num_doc):
    docs = []
    for i in range(num_doc):
        path = os.path.join(data_dir, "%d.txt" % i)
        with open(path) as f:
            docs.append(f.read())
    return docs

In [3]:
train_tags = parse_descriptions("cs5785-fall19-final/tags_train", num_doc=10000)
test_tags = parse_descriptions("cs5785-fall19-final/tags_test", num_doc=2000)

In [4]:
def parse_tags(tags):
    result = []
    for doc in tags:
        doc = doc.strip('\n').split('\n')
        cat_it = []
        if doc[0] == '':
            result.append('no tag')
        else:
            for tag in doc:
                split_tag = tag.split(':')
                cat_it.append(split_tag[0])
                cat_it.append(split_tag[1])
            parsed = (' ').join(list(cat_it))
            result.append(parsed)
    return np.array(result)

In [5]:
train_tags_parsed = parse_tags(train_tags)
test_tags_parsed = parse_tags(test_tags)

In [6]:
# function to preprocess data
def preprocessing(data):
    stop_words = set(stopwords.words('english')) # find stop words in English language
    lemmatizer = WordNetLemmatizer() # declare nltk lemmatizer

    # iterate through every sentence and replace it by itself lemmatized, without punctuation and without stop words
    for i in range(len(data)):
        sentence_no_punct = ''
        # remove punctuation
        
        for char in data[i]:
            if char not in string.punctuation:
                sentence_no_punct = sentence_no_punct + char
        data[i] = sentence_no_punct

        word_tokens = word_tokenize(data[i])
    
        # remove stop words and lemmatize
        word_tokens = [lemmatizer.lemmatize(word) for word in word_tokens if word not in stop_words and len(word) > 1]
        word_tokens = [lemmatizer.lemmatize(word, 'v') for word in word_tokens]
        word_tokens = [lemmatizer.lemmatize(word, 'a') for word in word_tokens]
        
        # remove conjunction words
        word_tokens = [word for word in word_tokens if word[-2:] != 'nt']
        (data[i]) = ' '.join(word_tokens)
        
    return data

In [7]:
train_descs = parse_descriptions('cs5785-fall19-final/descriptions_train', 10000)
test_descs  = parse_descriptions('cs5785-fall19-final/descriptions_test', 2000)
train_descs = preprocessing(train_descs)
test_descs  = preprocessing(test_descs)

#### Create the Bag of Words

In [8]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format("../GoogleNews-vectors-negative300.bin.gz", binary=True)

In [9]:
def doc_to_vec(sentence, word2vec):
    # get list of word vectors in sentence
    word_vecs = [word2vec.get_vector(w) for w in sentence.split() if w in word2vec.vocab]
    # return average
    return np.stack(word_vecs).mean(0)

In [10]:
x_train = np.array([doc_to_vec(train_descs[i], word2vec) for i in range(len(train_descs))])
x_test = np.array([doc_to_vec(d, word2vec) for d in test_descs])

#### Get Image Feature Vectors

In [11]:
train_feat_sort = pd.read_csv("sorted_train_features.csv", header = None, index_col=None)
test_feat_sort = pd.read_csv("sorted_test_features.csv", header = None, index_col=None)

#train_inter_feat = pd.read_csv("cs5785-fall19-final/features_train/features_resnet1000intermediate_train.csv", header = None, index_col=None)
#test_inter_feat = pd.read_csv("cs5785-fall19-final/features_test/features_resnet1000intermediate_test.csv", header = None, index_col=None)

In [12]:
#train_feat_2 = train_feat

#for i in range(1000):
    #train_feat[0][i] = train_feat[0][i].replace("images_train/", "").replace(".jpg", "")
    #train_feat[0][i] = temp

#train_feat_sort = train_feat.sort_values(by=0)

#for i in range(len(test_feat[0])):
 #   test_feat[0][i] = int(test_feat[0][i].replace("images_test/", "").replace(".jpg", ""))

#test_feat_sort = test_feat.sort_values(by=0)

In [13]:
#train_feat_sort = train_feat.sort_values(by=0)

In [14]:
#for i in range(len(test_feat[0])):
  #  test_feat[0][i] = int(test_feat[0][i].replace("images_test/", "").replace(".jpg", ""))

In [15]:
#y_train = train_feat_sort.drop(columns=0).to_numpy()
#y_test = test_feat_sort.drop(columns=0).to_numpy()

In [16]:
y_train = train_feat_sort.drop(columns=0).to_numpy()
y_test = test_feat_sort.drop(columns=0).to_numpy()
#y_train_inter = train_inter_feat_sort.drop(columns=0).to_numpy()
#y_test_inter = test_inter_feat_sort.drop(columns=0).to_numpy()

#### Create A Train/Test Split

In [17]:
import random
num_train = 8000
num_dev = 2000
num_test = 2000
split = list(range(num_train + num_dev))
random.shuffle(split)

In [18]:
x_train_dev = x_train[split[:num_train]]
y_train_dev = y_train[split[:num_train]]
x_test_dev = x_train[split[num_train:]]
y_test_dev = y_train[split[num_train:]]

train_tags_dev = train_tags_parsed[split[:num_train]]
test_tags_dev = train_tags_parsed[split[num_train:]]

#### For Later when we need the tags of the whole train and test set

In [19]:
tag_train = np.array([doc_to_vec(d, word2vec) for d in train_tags_parsed])
tag_test = np.array([doc_to_vec(d, word2vec) for d in test_tags_parsed])

#### For Development Right Now when we need the smaller train and test set to check accuracy

In [20]:
train_tags_dev = np.array([doc_to_vec(d, word2vec) for d in train_tags_dev])
test_tags_dev = np.array([doc_to_vec(d, word2vec) for d in test_tags_dev])

### Relate Descriptions and Tags with a Model --- SVM

In [38]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
supvm = MultiOutputRegressor(SVR(gamma = 'scale')).fit(x_train_dev, train_tags_dev)
tag_preds = supvm.predict(x_test_dev)

KeyboardInterrupt: 

### Relate Image Features and Descriptions Model

In [ ]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(solver='sgd', alpha=1e-5, hidden_layer_sizes=(300,), random_state=1, max_iter = 1000)
clf.fit(x_train_dev, y_train_dev)
image_preds = clf.predict(x_test_dev)

### Lasso Regressor with Cross-Validation to relate descriptors and tags (maybe try instead of mlp too?)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

lasso = Lasso()
parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv = 5)
lasso_regressor.fit(x_train_dev, train_tags_dev)
tag_preds = lasso_regressor.predict(x_test_dev)

print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.81904411315918, tolerance: 0.007436899468302727
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13.057409286499023, tolerance: 0.009094561450183392
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.5729217529296875, tolerance: 0.006425668019801378
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dua

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.751392841339111, tolerance: 0.0036355589982122183
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.403444290161133, tolerance: 0.00462539866566658
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.723752498626709, tolerance: 0.006132913287729025
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dual

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.46765422821045, tolerance: 0.0056828991509974
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.825466632843018, tolerance: 0.003891736501827836
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.4051642417907715, tolerance: 0.003128968644887209
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dualit

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.173499584197998, tolerance: 0.0026460126973688602
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.461452007293701, tolerance: 0.003317359136417508
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.579168796539307, tolerance: 0.004217358306050301
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dua

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.949646949768066, tolerance: 0.0037104820366948843
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.9861531257629395, tolerance: 0.0035046248231083155
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.731490612030029, tolerance: 0.0058217537589371204
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. 

...

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.317142486572266, tolerance: 0.005225296597927809
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.027231216430664, tolerance: 0.00829670112580061
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11.961868286132812, tolerance: 0.006497332826256752
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dual

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.363941192626953, tolerance: 0.005794820841401815
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.847479343414307, tolerance: 0.003339809365570545
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.891735553741455, tolerance: 0.0034589781425893307
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dua

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.081477642059326, tolerance: 0.0032355254516005516
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.308372497558594, tolerance: 0.004600978922098875
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.883215427398682, tolerance: 0.005584641359746456
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dua

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.6122465133667, tolerance: 0.006515826098620892
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.618003845214844, tolerance: 0.003944161348044872
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.801544666290283, tolerance: 0.004065935965627432
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dualit

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.364080905914307, tolerance: 0.0037330149207264185
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.219460487365723, tolerance: 0.005501404404640198
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.621907711029053, tolerance: 0.005123653449118137
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dua

### Ridge Regressor with Cross-Validation (not run yet)

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge()
parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}

ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv = 5)
ridge_regressor.fit(x_train_dev, y_train_dev)

print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

### Combine the Models

#### Try Evaluating the Nearest Neighbors and Take A New Ranking Based on Both

In [ ]:
# function to return the number of nearest neighbors specified
def knn_function(data, point, neighbors_number):
    knn = NearestNeighbors(n_neighbors=neighbors_number)
    knn.fit(data)
    return knn.kneighbors(point, neighbors_number, return_distance=False)

In [ ]:
tag_neighbors = knn_function(test_tags_dev, tag_preds, 50)
image_neighbors = knn_function(y_test_dev, image_preds, 50)

In [ ]:
neighbors1 = []
for i in range(len(tag_neighbors)):
    neighbors1.append(np.array(list(set(tag_neighbors[i]) & set(image_neighbors[i]))[:20]))

In [ ]:
neighbors1 = np.array(neighbors1)

### Evaluate the Model

In [ ]:
def scores(i):
    return (20+1-(i+1))/20

In [ ]:
def eval_accuracy(ytest, neighbors):
    # EVALUATE THE MODEL USING THE MEAN AVERAGE PRECISION AT 20
    scs = []
    for i in range(len(neighbors)):
        good = False
        for j, n in enumerate(neighbors[i]):
            if i == n:
                scs.append(scores(j))
                good = True
                break
        if good == False:
            scs.append(0)
    return sum(scs)/len(neighbors)

In [398]:
#print('accuracy of svm tags', eval_accuracy(y_test_dev, tag_neighbors[:20]))

accuracy of svm tags 0.1375


In [ ]:
print('accuracy of lasso tags', eval_accuracy(y_test_dev, tag_neighbors[:20]))

In [399]:
print('accuracy of image features', eval_accuracy(y_test_dev, image_neighbors[:20]))

accuracy of image features 0.1675


In [400]:
#print('accuracy of svm tags combined with image features', eval_accuracy(y_test_dev, neighbors1))

accuracy of svm tags combined with image features 0.323625


In [ ]:
print('accuracy of lasso tags combined with image features', eval_accuracy(y_test_dev, neighbors1))